In [41]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [43]:
import json
import json
import os
import sys
import shutil
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))
from segment.data import input_fn
from segment.display import WritePredictions
from networks.unet import unet_model

In [34]:
record_dir = '../cocorecord'
model_dir = '../trainings/unetcoco'
savedmodel = '../saved_model'
loadsavedmodel = None
trainingsetDescriptionFile = '{}/description.json'.format(record_dir)
trainingsetDescription = json.load(open(trainingsetDescriptionFile))
savedmodelname = '{}-dl3'.format(datetime.today().strftime('%Y-%m-%d-%H-%M-%S'))

In [65]:
config = {
        'clean_model_dir':True,
        'batch_size': 8,
        'trainingset': trainingsetDescription,
        'input_shape': [512, 512, 3],
        'classScale': 0.001, # scale value for each product class
        'augment_rotation' : 5., # Rotation in degrees
        'augment_flip_x': False,
        'augment_flip_y': True,
        'augment_brightness':0.,
        'augment_contrast': 0.,
        'augment_shift_x': 0.0, # in fraction of image
        'augment_shift_y': 0.0, # in fraction of image
        'scale_min': 0.75, # in fraction of image
        'scale_max': 1.25, # in fraction of image
        'ignore_label': trainingsetDescription['classes']['ignore'],
        'classes': trainingsetDescription['classes']['classes'],
        'image_crops': 1,
        'epochs': 3,
        'area_filter_min': 25,
        'validation_steps':25
        }

In [53]:
def WriteDictJson(outdict, path):

    jsonStr = json.dumps(outdict, sort_keys=False)
    f = open(path,"w")
    f.write(jsonStr)
    f.close()
       
    return True

In [54]:
if config['clean_model_dir']:
    shutil.rmtree(model_dir, ignore_errors=True)

In [55]:
model = LoadModel(model_dir, loadsavedmodel)

# Display model
model.summary()

Unable to load weghts from ../trainings/unetcoco
Unable to load weghts.  Restart training.
9412608/9406464 [==============================] - 0s 0us/step
Model: "UnetSegmentation"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
model (Model)                   [(None, 256, 256, 96 1841984     input_2[0][0]                    
__________________________________________________________________________________________________
sequential (Sequential)         (None, 32, 32, 512)  1476608     model[1][4]                      
__________________________________________________________________________________________________
concatenate (Concatenate)   

In [56]:
train_dataset = input_fn(True, record_dir, config)
test_dataset = input_fn(False, record_dir, config)

In [60]:
save_callback = tf.keras.callbacks.ModelCheckpoint(filepath=model_dir,verbose=1,save_weights_only=False,save_freq='epoch')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=model_dir, histogram_freq=1)

outpath = '{}/{}/'.format(savedmodel, savedmodelname)

if not os.path.exists(outpath):
    os.makedirs(outpath)

In [66]:
model_history = model.fit(train_dataset, epochs=config['epochs'],
                          steps_per_epoch=int(trainingsetDescription['sets'][0]['length']/config['batch_size']),
                          validation_steps=config['validation_steps'],
                          validation_data=test_dataset,
                          callbacks=[save_callback, tensorboard_callback])

Epoch 1/3
625/625 [==============================] - ETA: 0s - loss: nan - logits_loss: nan - logits_accuracy: 0.9978 - tf_op_layer_segmentation_accuracy: 0.6334
Epoch 00001: saving model to ../trainings/unetcoco
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../trainings/unetcoco/assets


InvalidArgumentError: Nan in summary histogram for: sequential_6/conv2d_transpose/kernel_0 [Op:WriteHistogramSummary]

In [ ]:
%tensorboard --logdir model_dir

In [ ]:
history = model_history.history
if 'loss' in history:
    loss = model_history.history['loss']
else:
    loss = []
if 'val_loss' in history:
    val_loss = model_history.history['val_loss']
else:
    val_loss = []

model_description = {'config':config,
                     'results': history
                    }
epochs = range(config['epochs'])

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.savefig('{}training.svg'.format(outpath))


WriteDictJson(model_description, '{}descrption.json'.format(outpath))
model.save(outpath)